# 🚀 GRPO v8: 강화학습 번역 모델 학습

LFM2-1.2B 모델을 COMET 보상 함수를 사용하여 GRPO로 학습합니다.

> **환경**: Google Colab T4 GPU

## 사용 전 설정
1. Colab Secrets에 `HF_TOKEN` 등록 (권장)
2. 데이터셋 파일 업로드 (`dataset_rl_final_10k.jsonl`)
2. `OUTPUT_MODEL_ID`를 본인 HuggingFace ID로 변경

In [ ]:
# 1. 패키지 설치
!pip uninstall -y numpy
!pip install "numpy>=1.26.0,<2.0.0"
!pip install -q transformers peft accelerate bitsandbytes
!pip install -q unbabel-comet datasets
!pip install -q git+https://github.com/huggingface/trl.git

print("✅ 설치 완료. [런타임] -> [세션 다시 시작] 클릭")

In [ ]:
# 2. HuggingFace 로그인
from huggingface_hub import login
login()  # Colab Secrets 사용 또는 수동 입력

In [ ]:
# 3. 라이브러리 임포트
import os
os.environ['WANDB_DISABLED'] = 'true'

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainerCallback
)
from trl import GRPOTrainer, GRPOConfig
from peft import LoraConfig, prepare_model_for_kbit_training
from comet import download_model, load_from_checkpoint

print(f"🔧 PyTorch: {torch.__version__}")
print(f"🔧 CUDA: {torch.cuda.is_available()}")

In [ ]:
# 4. 설정 (⚠️ OUTPUT_MODEL_ID는 본인 ID로 변경!)
MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6.4-merged"
OUTPUT_MODEL_ID = "YOUR_ID/your-output-model"  # ⚠️ 변경 필요!
DATASET_REPO = "gyung/lfm2-koen-samples"
DATASET_CONFIG = "manual_1000_grpo"  # GRPO용 데이터셋

print(f"📌 Base Model: {MODEL_ID}")
print(f"📌 Dataset: {DATASET_REPO} ({DATASET_CONFIG})")
print(f"📌 Dataset: {DATASET_REPO} ({DATASET_CONFIG})")
print(f"📌 Output: {OUTPUT_MODEL_ID}")

In [ ]:
# 5. COMET 심사 모델 로드
print("⚖️ 심사위원(COMET) 로딩 중...")
comet_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)

if torch.cuda.is_available():
    try:
        comet_model = comet_model.to("cuda")
        print("✅ COMET 모델 GPU 로드 완료")
    except Exception as e:
        print(f"⚠️ COMET GPU 로드 실패, CPU 사용: {e}")

In [ ]:
# 6. 데이터셋 준비 (HuggingFace에서 로드)
print(f"📚 데이터셋 로딩: {DATASET_REPO} ({DATASET_CONFIG})")
dataset = load_dataset(DATASET_REPO, DATASET_CONFIG, split="train")
print(f"   샘플 수 (양방향 포함): {len(dataset)}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def create_bidirectional_prompts(batch):
    """
    하나의 쌍(Input/Output)을 두 개의 학습 데이터로 변환
    1. En -> Ko
    2. Ko -> En
    """
    new_prompts = []
    new_srcs = []
    new_refs = []

    for en, ko in zip(batch['input'], batch['output']):
        # 1. En -> Ko
        msgs_en_ko = [
            {"role": "system", "content": "Translate the following text to Korean."},
            {"role": "user", "content": en}
        ]
        prompt_en = tokenizer.apply_chat_template(msgs_en_ko, tokenize=False, add_generation_prompt=True)
        new_prompts.append(prompt_en)
        new_srcs.append(en)
        new_refs.append(ko)

        # 2. Ko -> En
        msgs_ko_en = [
            {"role": "system", "content": "Translate the following text to English."},
            {"role": "user", "content": ko}
        ]
        prompt_ko = tokenizer.apply_chat_template(msgs_ko_en, tokenize=False, add_generation_prompt=True)
        new_prompts.append(prompt_ko)
        new_srcs.append(ko)
        new_refs.append(en)

    return {"prompt": new_prompts, "src": new_srcs, "ref": new_refs}

dataset = dataset.map(create_bidirectional_prompts, batched=True, remove_columns=dataset.column_names)

print(f"✅ 양방향 데이터셋 준비 완료")
print(f"   최종 샘플 수: {len(dataset)} (원본의 2배)")

In [ ]:
# 7. 보상 함수 정의 (COMET 기반)
def reward_function(completions, src, ref, **kwargs):
    """
    COMET 기반 보상 함수
    TRL 최신 API: dataset의 컬럼명(src, ref)이 키워드 인자로 전달됨
    """
    inputs = []
    for s, mt, r in zip(src, completions, ref):
        # completions가 dict인 경우 (conversational format)
        if isinstance(mt, dict):
            mt_text = mt.get('content', str(mt))
        elif isinstance(mt, list) and len(mt) > 0:
            mt_text = mt[-1].get('content', str(mt[-1])) if isinstance(mt[-1], dict) else str(mt[-1])
        else:
            mt_text = str(mt)

        inputs.append({"src": s, "mt": mt_text, "ref": r})

    with torch.no_grad():
        gpus = 1 if torch.cuda.is_available() else 0
        model_output = comet_model.predict(inputs, batch_size=8, gpus=gpus)

    return model_output.scores

print("✅ 보상 함수 정의 완료")

In [ ]:
# 8. 모델 로드 (4bit 양자화)
print("🤖 모델 로딩 중...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=False
)

# Norm 레이어 정밀도 조정
for name, module in model.named_modules():
    if "norm" in name.lower():
        module = module.to(torch.float32)

model.config.torch_dtype = torch.float16
model.config.use_cache = False

model = prepare_model_for_kbit_training(model)

print(f"✅ 모델 로드 완료: {MODEL_ID}")

In [ ]:
# 9. LoRA 및 학습 설정
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = GRPOConfig(
    output_dir="./lfm2-rl-final",

    # 학습 설정
    learning_rate=2e-6,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=5,

    # GRPO 설정
    num_generations=4,
    max_completion_length=200,
    beta=0.04,
    use_vllm=False,

    # 정밀도
    fp16=True,
    bf16=False,

    # 로깅 및 저장
    logging_steps=1,
    save_steps=50,
    save_total_limit=3,
    save_strategy="steps",
    report_to="none",

    # Hub 업로드
    push_to_hub=True,
    hub_model_id=OUTPUT_MODEL_ID,
    hub_strategy="every_save",
)

print("✅ 학습 설정 완료")

In [ ]:
# 10. Trainer 생성 및 학습
trainer = GRPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    reward_funcs=reward_function,
    peft_config=peft_config,
)

print("🚀 학습 시작...")
trainer.train()

In [ ]:
# 11. 모델 저장 및 업로드
trainer.save_model("./final_model")
trainer.push_to_hub()
print(f"✅ 업로드 완료: {OUTPUT_MODEL_ID}")